## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import csv
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,35.71,69,0,9.22,clear sky
1,1,Ribeira Grande,PT,38.5167,-28.7000,74.32,87,51,5.79,broken clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,33.46,86,75,14.97,light snow
3,3,Arlit,NE,18.7369,7.3853,93.47,27,91,6.20,overcast clouds
4,4,Boguchany,RU,58.3814,97.4531,57.52,89,75,4.59,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Myanaung,MM,18.2833,95.3167,75.94,91,100,3.91,overcast clouds
8,8,Ponta Do Sol,PT,32.6667,-17.1000,75.43,69,3,2.66,clear sky
10,10,Butaritari,KI,3.0707,172.7902,81.39,80,25,15.86,light rain
11,11,Jamestown,US,42.0970,-79.2353,77.72,81,75,0.00,broken clouds
17,17,Puerto Ayora,EC,-0.7393,-90.3518,78.76,87,78,5.99,broken clouds
19,19,Maunabo,PR,18.0072,-65.8993,84.02,80,5,11.90,clear sky
26,26,Bayamo,CU,20.3792,-76.6433,80.87,76,100,9.22,overcast clouds
28,28,Prainha,BR,-1.8000,-53.4800,79.56,70,67,2.48,broken clouds
32,32,Albany,US,42.6001,-73.9662,85.08,48,19,3.40,few clouds
33,33,Carupano,VE,10.6678,-63.2585,78.08,82,5,10.94,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                237
City                   237
Country                237
Lat                    237
Lng                    237
Max Temp               237
Humidity               237
Cloudiness             237
Wind Speed             237
Current Description    237
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Myanaung,MM,75.94,overcast clouds,18.2833,95.3167,
8,Ponta Do Sol,PT,75.43,clear sky,32.6667,-17.1000,
10,Butaritari,KI,81.39,light rain,3.0707,172.7902,
11,Jamestown,US,77.72,broken clouds,42.0970,-79.2353,
17,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
19,Maunabo,PR,84.02,clear sky,18.0072,-65.8993,
26,Bayamo,CU,80.87,overcast clouds,20.3792,-76.6433,
28,Prainha,BR,79.56,broken clouds,-1.8000,-53.4800,
32,Albany,US,85.08,few clouds,42.6001,-73.9662,
33,Carupano,VE,78.08,clear sky,10.6678,-63.2585,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
  # Grab the first hotel from the results and store the name.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df.count()

City                   237
Country                237
Max Temp               237
Current Description    237
Lat                    237
Lng                    237
Hotel Name             237
dtype: int64

In [10]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Myanaung,MM,75.94,overcast clouds,18.2833,95.3167,Taung Hlay Khar
8,Ponta Do Sol,PT,75.43,clear sky,32.6667,-17.1000,Hotel do Campo
10,Butaritari,KI,81.39,light rain,3.0707,172.7902,Isles Sunset Lodge
11,Jamestown,US,77.72,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
17,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file=("../Vacation_Search/WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))